In [12]:
import os.path as osp
import joblib
import pandas as pd

from sklearn.metrics import pairwise_distances
from bertopic import BERTopic

output_dir = osp.join('..', 'Output')

vectoriser = joblib.load(osp.join(output_dir, 'tfidf_vec.pkl'))
kmean_model = joblib.load(osp.join(output_dir, 'KMeans', 'kmean_model.pkl'))
bertopic_model = BERTopic.load(osp.join(output_dir, 'BERTopic', 'model'))

clustered_docs = {'KMeans': pd.read_csv(osp.join(output_dir, 'KMeans', 'clustered.csv'),
                                        encoding='utf-8'),
                  'BERTopic': pd.read_csv(osp.join(output_dir, 'BERTopic', 'clustered.csv'),
                                        encoding='utf-8')}
features = clustered_docs['KMeans'].drop(['cluster', 'link'], axis=1)

In [17]:
from typing import Iterable, Literal
def recommend(prompt: str, top_n: int = 5, *,
              model: Literal['BERTopic', 'KMeans'] = 'KMeans') -> Iterable[str]:
    if not isinstance(prompt, str):
        raise ValueError("prompt needs to be str")
    if not isinstance(top_n, int):
        raise ValueError("top_n needs to be an integer")

    temp = vectoriser.transform([prompt])
    promt_df = pd.DataFrame(temp.toarray(), columns=vectoriser.get_feature_names_out())
    
    if model == 'KMeans':
        mask = (clustered_docs[model]['cluster'] == kmean_model.predict(promt_df)[0])
    else:
        mask = (clustered_docs[model]['cluster'] == bertopic_model.transform(prompt)[0][0])
    
    dist_mat = pairwise_distances(promt_df, features[mask], metric='cosine')
    sorted_indices = dist_mat[0].argsort()

    if top_n > len(sorted_indices) or top_n < 1:
        top_n = len(sorted_indices)
    return clustered_docs[model]['link'].iloc[sorted_indices[-1:-(top_n + 1):-1]].values

In [20]:
prompt = "Vào lúc 18h30 ngày 24/8, Man Utd sẽ có chuyến làm khách tới sân Amex của Brighton ở vòng 2 giải Ngoại hạng Anh. Trước thềm trận đấu này, đoàn quân của HLV Ten Hag không được đánh giá cao bằng chủ nhà. Theo phân tích của siêu máy tính Opta (dựa theo 10.000 lần mô phỏng trận đấu), Brighton có 39,4% cơ hội giành chiến thắng. \
Tỷ lệ thắng của Man Utd chỉ là 36%. Khả năng trận đấu kết thúc với tỷ số hòa là 24,6%. Brighton có tỷ lệ thắng cao hơn Man Utd theo phân tích của siêu máy tính Opta (Ảnh: Opta). Thống kê trước trận Man Utd vs Brighton Man Utd đã giành chiến thắng 1-0 trong trận ra quân Ngoại hạng Anh mùa giải này gặp Fulham. Trong đó, tân binh Joshua Zirkzee là người hùng của Quỷ đỏ với bàn thắng ở phút 87. Theo thống kê, tiền đạo người Hà Lan trở thành cầu thủ đầu tiên vào sân và ghi bàn quyết định ngay trong trận ra mắt Man Utd kể từ Federico Macheda vào tháng 4/2009. \
Cầu thủ gần nhất ghi 2 bàn trong 2 trận đầu tiên cho Man Utd chính là Antony vào tháng 10/2022. Nếu chọc thủng lưới Brighton trong trận đấu hôm nay, Joshua Zirkzee sẽ có cơ hội tái hiện điều đó. \
Cầu thủ người Hà Lan sẽ được hỗ trợ bởi Bruno Fernandes, người đã đặt dấu giày trong 6/13 bàn thắng gần nhất của Man Utd (ghi 4 bàn, kiến tạo 2 bàn) trong các trận đấu gặp Brighton ở Ngoại hạng Anh. \
Man Utd đang hướng tới lần đầu tiên giành chiến thắng trong 2 trận đấu đầu tiên ở Ngoại hạng Anh kể từ mùa giải 2017/18. \
Tuy nhiên, họ sẽ phải đụng độ với đối thủ cứng đầu thực sự mang tên Brighton. Trong trận ra quân, Brighton đã giành chiến thắng 3-0 trước Everton nhờ các bàn thắng của Kaoru Mitoma, Danny Welbeck và Simon Adingra. \
Cựu tiền đạo Man Utd, Welbeck, đã ghi bàn trong 16 mùa giải Ngoại hạng Anh. Chỉ có 6 cầu thủ làm được tốt hơn thế là Ryan Giggs (ghi bàn ở 21 mùa), Paul Scholes (19), Frank Lampard (18), John Terry (17), Jermain Defoe (17), James Milner (17). \
Welbeck cũng ghi 4 bàn vào lưới Man Utd trong khuôn khổ Ngoại hạng Anh, giúp anh trở thành cầu thủ cũ ghi nhiều bàn vào lưới Quỷ đỏ nhất. \
James Milner có khả năng tiếp tục được trọng dụng bên cạnh Mats Wieffer ở hàng tiền vệ. Cầu thủ 38 tuổi này đã xuất hiện trong 23 mùa giải Ngoại hạng Anh khác nhau, nhiều nhất trong lịch sử giải đấu. \
James Milner đã thắng 11/35 trận đấu gặp Man Utd ở Ngoại hạng Anh, chỉ có Kyle Walker là cầu thủ giành nhiều chiến thắng hơn khi đối diện với Man Utd (12 bàn). \
Brighton là đội có tỷ lệ thắng cao nhất khi đối đầu với Man Utd ở Ngoại hạng Anh (Ảnh: Opta). \
Thành tích đối đầu Brighton vs Man Utd \
Brighton được xem là hung thần của Man Utd trong những năm qua. Theo thống kê, ""Chim mòng biển"" đã thắng 4/5 trận gần nhất gặp Quỷ đỏ ở Ngoại hạng Anh. Tuy nhiên, trong lần chạm trán gần nhất ở vòng đấu cuối cùng mùa giải trước, Brighton đã thua 0-2 trước Man Utd ngay trên sân nhà. Diogo Dalot và Hojlund là những người ghi bàn cho đội bóng thành Manchester. \
Điều đáng nói, Man Utd thất bại trong 43% trận đấu gặp Brighton (6/14 trận). Trong lịch sử giải Ngoại hạng Anh, Quỷ đỏ chưa thua đội bóng nào với tỷ lệ cao đến vậy. \
14 trận gặp nhau của hai đội tại Ngoại hạng Anh không chứng kiến kết quả hòa. Brighton thắng 6 trận và thua 8 trận trước Man Utd. \
Thông tin lực lượng Brighton vs Man Utd \
Brighton không có sự phục vụ của hàng loạt cầu thủ như Solly March, Bart Verbruggen, Evan Ferguson, Julio Enciso, Pervis Estupinan vì chấn thương. Bên cạnh đó, Billy Gilmour và Yankuba Minteh vẫn đang chạy đua với thời gian để kịp ra sân. \
Man Utd không có sự phục vụ dài hạn của Leny Yoro và Rasmus Hojlund. Bên cạnh đó, Tyrell Malacia chưa thể trở lại. Các cầu thủ Luke Shaw, Victor Lindelof, Jadon Sancho chưa chắc ra sân ở trận đấu này. \
Đội hình dự kiến Brighton vs Man Utd \
Brighton: Jason Steele, Jack Hinshelwood, Lewis Dunk, Joël Veltman, Mats Wieffer, James Milner, Kaoru Mitoma, Jõao Pedro, Simon Adingra, Danny Welbeck. \
Man Utd: André Onana, Diogo Dalot, Lisandro Martínez, Harry Maguire, Noussair Mazraoui, Casemiro, Kobbie Mainoo, Marcus Rashford, Bruno Fernandes, Alejandro Garnacho, Joshua Zirkzee. \
Dự đoán tỷ số Brighton 3-2 Man Utd \
Dự đoán của một vài tờ báo trên thế giới \
Whoscored: Brighton 2-2 Man Utd \
Sport Mole: Brighton 2-2 Man Utd \
Sportskeeda: Brighton 1-2 Man Utd \
Squawka: Brighton 2-0 Man Utd"
# prompt = "khoa học"
# prompt = "trương mỹ lan biển thủ ba chục tỷ đồng"

In [21]:
recommend(prompt, 10, model='BERTopic')

array(['https://vnexpress.net/bo-giao-duc-co-bat-cong-giua-cac-to-hop-xet-tuyen-dai-hoc-4810530.html',
       'https://vnexpress.net/mobifone-to-chuc-giai-dau-esports-cho-sinh-vien-4821972.html',
       'https://vnexpress.net/co-giao-nguoi-mong-tap-huan-stem-ai-cho-ca-huyen-4817222.html',
       'https://vnexpress.net/nam-sinh-top-5-dau-ra-bach-khoa-tp-hcm-co-diem-luan-van-9-9-4819400.html',
       'https://vnexpress.net/sinh-vien-tu-vong-khi-thuc-hanh-noi-dien-4822104.html',
       'https://vnexpress.net/dai-hoc-dau-tien-cong-bo-muc-thuong-tet-cho-giang-vien-4809840.html',
       'https://vnexpress.net/gan-50-dai-hoc-cong-bo-lich-nghi-tet-4815185.html',
       'https://vnexpress.net/10-cum-dong-tu-pho-bien-bat-dau-bang-take-4809761.html',
       'https://vnexpress.net/du-hoc-sinh-den-my-cao-nhat-moi-thoi-4817619.html',
       'https://vnexpress.net/ke-toan-van-thu-truong-hoc-mong-co-them-phu-cap-4815029.html'],
      dtype=object)

In [23]:
recommend(prompt, 10)

array(['https://vnexpress.net/de-tham-khao-8-mon-thi-danh-gia-nang-luc-dh-su-pham-ha-noi-4818998.html',
       'https://vnexpress.net/nha-vo-dich-olympia-bi-quyet-la-su-gan-li-va-chien-thuat-thong-minh-4803572.html',
       'https://vnexpress.net/dai-hoc-quoc-gia-australia-mo-rong-tuyen-hoc-sinh-viet-nam-4824518.html',
       'https://vnexpress.net/trung-quoc-se-tang-hoc-bong-cho-du-hoc-sinh-viet-nam-4826104.html',
       'https://vnexpress.net/truong-hoc-thao-tivi-tra-lai-tien-phu-huynh-4799540.html',
       'https://vnexpress.net/de-minh-hoa-8-mon-thi-v-sat-4814407.html',
       'https://vnexpress.net/dai-hoc-can-ket-noi-voi-truong-pho-thong-de-dao-tao-nhan-luc-4816692.html',
       'https://vnexpress.net/tan-sinh-vien-choang-vi-tuyen-cau-lac-bo-kho-nhu-xin-viec-4812668.html',
       'https://vnexpress.net/dai-hoc-quoc-gia-tp-hcm-bo-hai-cach-xet-tuyen-tu-nam-2025-4800873.html',
       'https://vnexpress.net/bo-giao-duc-co-dinh-mon-thi-lop-10-co-the-khien-hoc-sinh-hoc-tu-4801229.html'